In [ ]:
##### ESPACIO #####
# Importamos las librerías necesarias
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit, bisect
from scipy.interpolate import CubicSpline
import pandas as pd




In [ ]:
##### ESPACIO #####
V_t = np.array([0., 0.09997963, 0.19979541, 0.29818724, 0.38708739,
       0.44918412, 0.4858808 , 0.5092326 , 0.5257116 , 0.5384415 ,
       0.5484803 , 0.556853  , 0.5640389 , 0.5703152 , 0.5758803 ,
       0.5808764 ])

I = np.array([0., 2.036700e-08, 2.045907e-07, 1.813641e-06,
       1.291275e-05, 5.081596e-05, 1.141195e-04, 1.907880e-04,
       2.742902e-04, 3.617043e-04, 4.515953e-04, 5.431760e-04,
       6.359759e-04, 7.296934e-04, 8.241251e-04, 9.191271e-04])




In [ ]:
##### ESPACIO #####
V_out = np.array([0.000000e+00, 2.036700e-05, 2.045893e-04, 1.812759e-03,
       1.291261e-02, 5.081588e-02, 1.141192e-01, 1.907674e-01,
       2.742884e-01, 3.615585e-01, 4.515197e-01, 5.431470e-01,
       6.359611e-01, 7.296848e-01, 8.241197e-01, 9.191236e-01])

V_in = np.array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5])




In [ ]:
##### ESPACIO #####
def corriente_I(V, a, b):
    return a * (np.exp(b * V) - 1)




In [ ]:
##### ESPACIO #####
I_fit = curve_fit(corriente_I, V_t, I)
I_teo = corriente_I(V_t, I_fit[0][0], I_fit[0][1])

print('I_0 =', I_fit[0][0], 'T =', I_fit[0][1])

suma = 0
for i in range(len(V_t)):
    suma = suma + (I_teo[i] - I[i])**2

print('El error cuadrático es de', suma/len(V_t), '[mA]')
    
plt.scatter(V_t, I_teo, label = 'Ajuste', color = 'red')
plt.plot(V_t, I, label = 'Datos', color = 'blue')
plt.title('Ajuste de Voltaje vs. Corriente')
plt.xlabel('Voltaje [V]')
plt.ylabel('Corriente [mA]')
plt.legend()
plt.grid()
plt.savefig(r"C:\Users\Usuario\Documents\Astro experimental\Informe 3\graf1.png")
plt.show()




In [ ]:
##### ESPACIO #####
# Constantes
R = 1000
I0 = I_fit[0][0]
b = I_fit[0][1]

# voltaje de entrada y salida
V_out_fitted=np.zeros(len(V_in))

# funcion que se buscara ceros
def g(v_out):
    return R * I0 * (np.exp((V_in[i] - v_out) * b) - 1) - v_out

for i in range(len(V_in)):
    V_out_fitted[i] = bisect(g,10,-10)
    
print(V_out_fitted)




In [ ]:
##### ESPACIO #####
plt.plot(V_in, V_out_fitted, label='Ajuste', color = 'red')
plt.plot(V_in, V_out, 'o', label='Datos', color = 'blue')
plt.title('$V_{out}$ vs. $V_{in}$')
plt.xlabel('$V_{in}$ [V]')
plt.ylabel('$V_{out}$ [V]')
plt.legend()
plt.grid()
plt.savefig(r"C:\Users\Usuario\Documents\Astro experimental\Informe 3\graf2.png")
plt.show()




In [ ]:
##### ESPACIO #####
# pasamos a array
V_in = np.array(V_in)

# expandimos
Vind = np.append(V_in, [1.6, 1.7, 1.8])

# extrapolamos
cs = CubicSpline(V_in, V_out_fitted)

Voutd = cs(Vind)




In [ ]:
##### ESPACIO #####
lenght = len(V_out_fitted)
d1, d2, d3, d4 = np.zeros(lenght), np.zeros(lenght), np.zeros(lenght), np.zeros(lenght)

for i in range(len(V_out_fitted)-1):
    h = 0.1 # V_in[i + 1] - V_in[i] = 0.01 siempre
    d1[i + 1]  = (V_out_fitted[i + 1] - V_out_fitted[i]) / h
    d2[i + 1] = (d1[i + 1] - d1[i]) / h
    d3[i + 1] = (d2[i + 1] - d2[i]) / h
    d4[i + 1] = (d3[i + 1] - d3[i]) / h
   
d1[0], d2[0], d3[0], d4[0]  = 1e-5 , 1e-5, 1e-5 , 1e-5

print(d4)




In [ ]:
##### ESPACIO #####
plt.plot(V_in, V_out, label = 'Datos')
plt.plot(V_in, d1, label = '1ra der.')
plt.plot(V_in, d2, label = '2da der')
plt.plot(V_in, d3, label = '3ra der.')
plt.plot(V_in, d4, label = '4ta der.')
plt.legend()
plt.grid()
plt.savefig(r"C:\Users\Usuario\Documents\Astro experimental\Informe 3\graf3.png")
plt.show()




In [ ]:
##### ESPACIO #####
# Insertamos la fórmula del radio
R_curv = abs((( 1 + d1 ** 2) ** (3/2)) / d2)
index = list(R_curv).index(min(R_curv))
print('El radio de curvatura mínimo es',float(R_curv[index]), 'con un voltaje de entrada de',
      float(V_in[index]),'[V] y un voltaje de salida de', float(V_out[index]), '[V]')




In [ ]:
##### ESPACIO #####
X = 1/R_curv

plt.plot(V_in, X)
plt.title('$V_{in}$ vs. Curvatura de $V_{out}(V_{in})$')
plt.ylabel('Curvatura de $V_{out}(V_{in})$')
plt.xlabel('$V_{in}$ [V]')
plt.xlim(0,1.5)
plt.grid()
plt.savefig(r"C:\Users\Usuario\Documents\Astro experimental\Informe 3\graf4.png")
plt.show()




In [ ]:
##### ESPACIO #####
print('Podemos ver que la curvatura máxima se encuentra en un voltaje de entrada de', V_in[list(X).index(max(X))], '[V]')




In [ ]:
##### ESPACIO #####
P2nd = np.array([-87.346, -69.003, -53.25, -42.943,
                  -38.22, -37.82, -41.43, -47.002,
                  -52.583, -57.053, -60.750, -64.501,
                  -67.199, -70.060, -72.633, -74.421])

V_in_offset = np.array([0., .1, .2, .3, .4, .5, .6, .7, .8,
                     .9, 1, 1.1, 1.2, 1.3, 1.4, 1.5 ])




In [ ]:
##### ESPACIO #####
plt.plot(V_in_offset, P2nd)
plt.ylabel('Potencia [W]')
plt.xlabel('DC-offset [V]')
plt.title('Potencia en el segundo armónico')
plt.grid()
plt.savefig(r"C:\Users\Usuario\Documents\Astro experimental\Informe 3\graf5.png")
plt.show()




In [ ]:
##### ESPACIO #####
maxP = max(P2nd) # P = V \cdot I
maxDC = V_in_offset[list(P2nd).index(maxP)]

print('El valor de DC-offset donde se encuentra el',
      'máximo contenido armónico es', maxDC, '[V]',
      'con', maxP, '[W]')




In [ ]:
##### ESPACIO #####
data2 = pd.read_table(r"C:\Users\Usuario\Downloads\circuito2clean.csv", delimiter = ',')
frequencies = data2['Freq.'] 
vouts = data2['V(out)']

plt.plot(frequencies, vouts, linewidth = 0.5)
plt.xlim(0,20000)
plt.ylim(-150,-20)
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('$V_{out}$ [dB]')
plt.title('Espectro con DC-offset = 0.5 [V]')
plt.grid()
plt.savefig(r"C:\Users\Usuario\Documents\Astro experimental\Informe 3\graf6.png")
plt.show()




In [ ]:
##### ESPACIO #####
# Calculamos la frecuencia de los armonicos

armonicos = np.array([68.34, 12.86, 0.28, 0.51])




In [ ]:
##### ESPACIO #####
# Calculamos las derivadas en 0.5 [V]

V = V_out[5]
dV  = d1[5]
d2V = d2[5]
d3V = d3[5]
d4V = d4[5]




In [ ]:
##### ESPACIO #####
coef0 = V + d2V/4 + d4V/64
coef1 = d3V/8 + dV
coef2 = d2V/4 + d4V/48
coef3 = d3V/24
coef4 = d4V/192

coefs = np.array([coef1, coef2, coef3, coef4])
print(coefs)




In [ ]:
##### ESPACIO #####
data4 = pd.read_table(r"C:\Users\Usuario\Downloads\circuito4clean.csv", delimiter = ',')
frequencies2 = data4['Freq.']
vouts2 = data4['V(out)']
plt.plot(frequencies2, vouts2, linewidth = 0.5)
plt.xlim(0,20000)
plt.ylim(-150,-20)
plt.xlabel('Frecuencia [KHz]')
plt.ylabel('$V_{out}$ [dB]')
plt.grid()
plt.show()
